In [1]:
import io
import cv2
# import math
# import json
import base64 
import numpy as np
from PIL import Image
import pytesseract as tess
from scipy import ndimage as ndi
from matplotlib import pyplot as plt
import plotly.express as px

tess.pytesseract.tesseract_cmd = "C:\\Users\\alber\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe"

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
chrome_options = webdriver.ChromeOptions()
mobile_emulation = { "deviceName": "iPhone 6" }
chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
driver = webdriver.Chrome("C:\\Users\\alber\\Documents\\My_Code\\Primer_game\\chromedriver.exe", options= chrome_options)
driver.get('https://primerlearning.org/')

In [3]:
driver.set_window_size(300,850)
coin_flip_app = driver.find_element_by_id("coin-flip-app")
driver.execute_script("arguments[0].scrollIntoView();", coin_flip_app)

In [ ]:
driver.find_element_by_id("unity-loading-bar")

In [8]:
#iframe = driver.find_elements_by_tag_name('iframe')[0]
#driver.switch_to.frame(iframe)
# progress =  (WebDriverWait(driver, 20)
#             .until(EC.visibility_of_element_located((By.ID, "unity-progress-bar-full")))
#             )
# progress =  WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID,"unity-progress-bar-full")))
# progressCondition = progress.get_attribute('width')
# print(progressCondition)
# print(driver.find_element_by_id("unity-progress-bar-full").get_attribute("style"))
iframe = driver.find_elements_by_tag_name('iframe')[0]
driver.switch_to.frame(iframe)
progress = (WebDriverWait(driver, 40)
           .until(lambda drvr: (drvr.find_element_by_id("unity-progress-bar-full")
                                    .get_attribute("style") == "width: 100%;"))
           )
driver.switch_to.default_content()

In [ ]:
num = driver.execute_script('return document.querySelector("#unity-progress-bar-full");')

In [ ]:
# actions = webdriver.common.action_chains.ActionChains(driver)
# actions.move_to_element(coin_flip_app).perform()


In [ ]:
# link_bar = driver.find_element_by_class_name("link-bar")
# page_footer = driver.find_element_by_class_name("page-footer")
# page_content = driver.find_element_by_class_name("page-content")


In [ ]:
x, y = 330, 500
# Show flipping animation: x, y = 50, 550
# 5 Coins: x, y = 330, 500
# 1 Coin: x, y = 20, 500
#Cheater x, y = 330, 600
#Leaderboard x, y = 330, 20
#Fair: x, y = 20, 600
#Game: x, y = 20, 20
action = ActionChains(driver)
action.move_to_element_with_offset(coin_flip_app, x, y)
action.click()
action.perform()

In [ ]:
# Get the score
def get_page_screenshot():
    screenshot = driver.get_screenshot_as_base64()
    screenshot = base64.b64decode(screenshot)
    screenshot = io.BytesIO(screenshot)
    screenshot = Image.open(screenshot).convert('L')
    screenshot = np.asarray(screenshot)
    return screenshot

def crop_to_results(screenshot):
    return screenshot[490:600, 325:550]

def get_results(screenshot):
    text = tess.image_to_string(crop)
    if 'Heads: ' in text and 'Tails: ' in text:
        text_tuples = [line.split(': ') for line in text.split('\n')[:2]]
        text_dict = {k: int(v) for k, v in text_tuples}
        return text_dict
    return {'Heads': 0, 'Tails': 0}

image = get_page_screenshot()
crop = get_results(image)
plt.imshow(crop)

In [ ]:
thresholded_image = image.copy().astype(np.uint8)
import cv2
thresholded_image = cv2.GaussianBlur(thresholded_image, (5, 5), 9)
threshold = 255
thresholded_image[image>=threshold] = 255
thresholded_image[image<threshold] = 0
label_objects, nb_labels = ndi.label(thresholded_image)
print(nb_labels)

In [ ]:
label_objects, nb_labels = ndi.label(thresholded_image)
print(nb_labels)
a = label_objects.copy()
a[label_objects>0] = 0
a[label_objects==63] = 1

def remove_noise(label_img,threshold):
    label_img_new = label_img.copy()
    n_comp = np.amax(label_img)
    for i in range(n_comp+1):
        n_pixels = sum(sum(label_img==i))
        if n_pixels<threshold : 
             label_img_new[label_img==i] = 0
    return label_img_new

new = remove_noise(label_objects,100)
px.imshow(new)

In [ ]:
print(nb_labels)